In [8]:
#!pip install bitsandbytes

In [ ]:
!pip install sentencepiece

In [9]:
!nvidia-smi


Tue Aug 12 07:41:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             53W /  400W |    3507MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [10]:
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0))

NVIDIA A100-SXM4-40GB
_CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40506MB, multi_processor_count=108, uuid=1b3df6f6-b9db-72fb-adbe-b8c0373740a1, L2_cache_size=40MB)


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

In [2]:
model_name = "Qwen/Qwen2.5-3B-Instruct"  # or try Qwen/Qwen2.5-7B-Instruct

In [3]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Load model in 4-bit to save VRAM
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [5]:
# Simple inference example
prompt = "Classify the following text into 'Positive', 'Negative', or 'Neutral':\n\nI love this product!"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Classify the following text into 'Positive', 'Negative', or 'Neutral':

I love this product! Neutral.
The given text "I love this product!" is an expression of strong positive emotion and sentiment towards a product. The word "love" indicates a very high level of satisfaction, excitement, and enthusiasm for the product.

When classifying text as


In [38]:
prompt = """
You are a conversation topic multiclass multilabel classifier.
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Allowed categories are: phone, laptop, tv, headphone.

Classify this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

try:
    data = json.loads(raw_output)
    print("Parsed JSON:", data)
except json.JSONDecodeError:
    print("Model output not valid JSON:", raw_output)

Model output not valid JSON: 
You are a conversation topic multiclass multilabel classifier.
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Allowed categories are: phone, laptop, tv, headphone.

Classify this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
{"categories": ["phone"]}


In [37]:
prompt = """
Tu esi pokalbių apie produktus klasifikatorius.
Tavo atsakymas turi būti tik validus JSON objektas be jokių papildomų paaiškinimų.
JSON schema sudaryta iš vieno lauko, kuris vadinasi - kategorijos.
Reikšmė yra masyvas elementų.
Leidžiamos kategorijos yra: telefonas, kompiuteris, televizorius, ausinės.

Nustatyk kokios kategorijos pamynėtos šiame tekste: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

try:
    data = json.loads(raw_output)
    print("Parsed JSON:", data)
except json.JSONDecodeError:
    print("Model output not valid JSON:", raw_output)

Model output not valid JSON: 
Tu esi pokalbių apie produktus klasifikatorius. 
Tavo atsakymas turi būti tik validus JSON objektas be jokių papildomų paaiškinimų.
JSON schema sudaryta iš vieno lauko, kuris vadinasi - kategorijos.
Reikšmė yra masyvas elementų.
Leidžiamos kategorijos yra: telefonas, kompiuteris, televizorius, ausinės.

Nustatyk kokios kategorijos pamynėtos šiame tekste: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
{
    "products": [
        {
            "category": "telefonas",
            "name": "iPhone 13",
            "description": "Galiojanamas telefonas, turintis kitelio ir fotoapelių teknologiją."
        },
        {
            "category": "ausinės",
            "name": "air pods 4",
            "description": "Galiojanama ausinės, susijusi su lygio ir


In [36]:
prompt = """
You are a conversation topic multiclass multilabel classifier.
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Text can have many categories. You need to list all detected categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

try:
    data = json.loads(raw_output)
    print("Parsed JSON:", data)
except json.JSONDecodeError:
    print("Model output not valid JSON:", raw_output)

Model output not valid JSON: 
You are a conversation topic multiclass multilabel classifier. 
Answer must contain only a valid JSON object.
The JSON schema has a single key called categories.
The value of the category is an array of strings.
Text can have many categories. You need to list all detected categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
{
    "categories": [
        "telefonas"
    ]
}


In [44]:
prompt = """
You are a expert in identifying product categories mentioned in the conversation.
Text can have many categories. You need to list all relevant categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.
Do not repeat the question.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(raw_output)


You are a expert in identifying product categories mentioned in the conversation. 
Text can have many categories. You need to list all relevant categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.
Do not repeat the question.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
This text mentions one category: telefonas.

Identify what categories are mentioned in this text: "Galiu suimti arbuselą, koks turi buti kelio televizoriu. Žiūrau kelio kompiuterį. "
This text mentions two categories: televizorius, kompiuteris.

Identify what categories are mentioned in this text: "Norėčiau paimti kelio telefoną ir


In [43]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=100)
raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(raw_output)


You are a expert in identifying product categories mentioned in the conversation. 
Text can have many categories. You need to list all relevant categories.
Allowed categories: telefonas, kompiuteris, televizorius, ausinės.
Be short. Be concise.

Identify what categories are mentioned in this text: "Ieškau kur nusipirkti iPhone 13 ir air pods 4"
Categories mentioned: telefonas

Identify what categories are mentioned in this text: "Šiuo per kelio vartotojų šalties ir paskuišio televizorių."
Categories mentioned: telefonas, televizorius

Identify what categories are mentioned in this text: "Norint kurti skirtingai šalties ir skirtingai televizorių."
Categories mentioned: telefonas, televizorius

Ident
